<h1 align="center">3.3 Feature Engeneering (Sales)</h1>
<h3 align="center">Dataset: <a href="https://www.kaggle.com/c/competitive-data-science-predict-future-sales">Predict future sales</a></h3>

### Imports

In [1]:
import numpy   as np
import pandas  as pd
import seaborn as sb
import matplotlib.pyplot as plt

sb.set()
%matplotlib inline
#%matplotlib notebook

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics       import mean_squared_error
import lightgbm as lgb

from itertools import product
import calendar 
import gc
import time
import pickle

from ipywidgets import interact

### Constants

In [2]:
ENGLISH = True

DATA_RUS_PATH = "../DATA/1. Original data Russian (96Mb)/"
DATA_ENG_PATH = "../DATA/2. Translated data English (1Mb)/"
DATA_FE_PATH  = "../DATA/3. Feature Engeneering/"
DATA_SUB_PATH = "../DATA/5. Submissions/"

### Load data

In [3]:
sales = pd.read_csv(DATA_RUS_PATH + "sales_train.csv")          # Dayly sales  Jan 2013 -> Oct 2015
test  = pd.read_csv(DATA_RUS_PATH + "test.csv", index_col="ID") # Predict Noviember 2015
sub   = pd.read_csv(DATA_RUS_PATH + "sample_submission.csv", index_col="ID")

if ENGLISH: 
    shops = pd.read_csv(DATA_ENG_PATH + "shops.csv", index_col="shop_id") # shops    (60)
    items = pd.read_csv(DATA_ENG_PATH + "items.csv")           # products  (22170)
    cats  = pd.read_csv(DATA_ENG_PATH + "item_categories.csv") # product categories (84)

else:
    shops = pd.read_csv(DATA_RUS_PATH + "shops.csv", index_col="shop_id")           # shops    (60)
    items = pd.read_csv(DATA_RUS_PATH + "items.csv")           # products  (22170)
    cats  = pd.read_csv(DATA_RUS_PATH + "item_categories.csv") # product categories (84)

In [4]:
def sales_info():
    print("Number of shops:", sales.shop_id.nunique())
    print("Number of items:", sales.item_id.nunique())
    print("Number of sales:", len(sales))
sales_info()

Number of shops: 60
Number of items: 21807
Number of sales: 2935849


# <center> Data Cleaning

### Remove outliers

In [5]:
sales = sales.query('0 < item_price < 50000 and 0 < item_cnt_day < 1001')
sales_info()

Number of shops: 60
Number of items: 21802
Number of sales: 2928488


### Remove duplicates

In [6]:
duplicated_shops_replacement = {
    0: 57,
    1: 58,
    11: 10,
    40: 39 # Not sure about this one
}

def load_obj(name):
    with open(DATA_FE_PATH + name + '.pkl', 'rb') as f:
        return pickle.load(f)
    
duplicated_items_replacement = load_obj("duplicated_items_replacement")


sales = sales.replace({
        'shop_id': duplicated_shops_replacement, # replacing obsolete shop id's
        'item_id': duplicated_items_replacement  # fixing duplicate item id's  
})

sales_info()

Number of shops: 56
Number of items: 21356
Number of sales: 2928488


### Removing shops which don't appear in the test set

In [7]:
sales = sales[sales['shop_id'].isin(test.shop_id.unique())]
sales_info()

Number of shops: 42
Number of items: 20693
Number of sales: 2427355


# <center> Date information

### Convert to datatime format

In [8]:
sales['date'] = pd.to_datetime(sales.date,format='%d.%m.%Y')

### Extract time feats

In [9]:
sales['weekday'] = sales.date.dt.dayofweek

#first day the item was sold, day 0 is the first day of the training set period
sales['first_sale_day'] = sales.date.dt.dayofyear 
sales['first_sale_day'] += 365 * (sales.date.dt.year-2013)
sales['first_sale_day'] = sales.groupby('item_id')['first_sale_day'].transform('min').astype('int16')

# ALSO BY groupby('item_id', "shop_id")

In [10]:
sales

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,weekday,first_sale_day
0,2013-01-02,0,59,22154,999.00,1.0,2,2
1,2013-01-03,0,25,2552,899.00,1.0,3,3
3,2013-01-06,0,25,2554,1709.05,1.0,6,6
4,2013-01-15,0,25,2555,1099.00,1.0,1,15
5,2013-01-10,0,25,2564,349.00,1.0,3,5
...,...,...,...,...,...,...,...,...
2935844,2015-10-10,33,25,7409,299.00,1.0,5,1003
2935845,2015-10-09,33,25,7460,299.00,1.0,4,761
2935846,2015-10-14,33,25,7459,349.00,1.0,2,2
2935847,2015-10-22,33,25,7440,299.00,1.0,3,803


In [11]:
tr = set(sales.shop_id.unique())
te = set(test.shop_id.unique())

print("Train shops:", len(tr))
print("Test shops: ", len(te))
print()
print("Shops only in train:", len(tr-te))
print("Same shops at train test:",  len(tr & te))
print("Shops only in test:",  len(te-tr))

Train shops: 42
Test shops:  42

Shops only in train: 0
Same shops at train test: 42
Shops only in test: 0


In [12]:
print(shops.shop_name[0],  "==", shops.shop_name[57])
print(shops.shop_name[1],  "==", shops.shop_name[58])
print(shops.shop_name[10], "==", shops.shop_name[11])
print(shops.shop_name[39], "==", shops.shop_name[40])

Yakutsk Ordzhonikidze - 56 francs == Yakutsk Ordzhonikidze - 56
Yakutsk TC Central fran == Yakutsk TC Central 
Zhukovsky st. Chkalov 39m? == Zhukovsky st. Chkalov 39m²
Rostov-on-Don SEC Megacenter Horizon == Rostov-on-Don Megatsentr Gorizont Ostrovnoy shopping mall


In [13]:
sales.loc[sales.shop_id == 0,  'shop_id'] = 57  # Якутск Орджоникидзе, 56
sales.loc[sales.shop_id == 1,  'shop_id'] = 58  # Якутск ТЦ "Центральный"
sales.loc[sales.shop_id == 11, 'shop_id'] = 10  # Жуковский ул. Чкалова 39м²  # CORREGIDO
sales.loc[sales.shop_id == 40, 'shop_id'] = 39

In [14]:
tr = set(sales.shop_id.unique())
te = set(test.shop_id.unique())

print("Train shops:", len(tr))
print("Test shops: ", len(te))
print()
print("Shops only in train:", len(tr-te))
print("Same shops at train test:",  len(tr & te))
print("Shops only in test:",  len(te-tr))

Train shops: 42
Test shops:  42

Shops only in train: 0
Same shops at train test: 42
Shops only in test: 0


# <center> PART 2: FEATURE ENGINEERING

# Tratamiento de Categorías: Sacar tipo y subtipo

In [15]:
cats.head() #Аксессуары - PS2 --> Accesorios PS2

,item_category_name,item_category_id
0,PC - Headsets / Headphones,0
1,Accessories - PS2,1
2,Accessories - PS3,2
3,Accessories - PS4,3
4,Accessories - PSP,4


In [16]:
split = cats['item_category_name'].str.split('-') # dividir el nombre de la categoría por '-'

cats['item_type']    = split.map(lambda x: x[0].strip()) # primera parte tipo de categoría
cats['item_subtype'] = split.map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip()) #segunda parte subtipo
cats.head()

,item_category_name,item_category_id,item_type,item_subtype
0,PC - Headsets / Headphones,0,PC,Headsets / Headphones
1,Accessories - PS2,1,Accessories,PS2
2,Accessories - PS3,2,Accessories,PS3
3,Accessories - PS4,3,Accessories,PS4
4,Accessories - PSP,4,Accessories,PSP


In [17]:
# Codificar
cats['item_type']    = LabelEncoder().fit_transform(cats['item_type'])
cats['item_subtype'] = LabelEncoder().fit_transform(cats['item_subtype'])
cats = cats[['item_category_id', 'item_type', 'item_subtype']]
cats.head()

,item_category_id,item_type,item_subtype
0,0,13,38
1,1,0,47
2,2,0,48
3,3,0,49
4,4,0,51


# Tratamiento de Productos

In [18]:
items.item_id.nunique(), len(items)

(22170, 22170)

In [19]:
items.head(20)

,item_id,item_category_id,item_name
0,0,40,!! IN THE POWER OF HAPPINESS (PLAST) D
1,1,76,! ABBYY FineReader 12 Professional Edition Ful...
2,2,40,*** IN THE GLORY OF THE GLORY (UNV) D
3,3,40,*** BLUE WAVE (Univ) D
4,4,40,*** BOX (GLASS) D
5,5,40,*** NEW AMERICAN GRAPHICS (UNI) D
6,6,40,*** BLOCK ON THE GATES (UNI) D
7,7,40,*** SHOCK AT DOOR-2 (UNI) D
8,8,40,*** TEA WITH MUSSOLINI D
9,9,40,*** SHUGARLEND EXPRESS (UNI) D


In [20]:
# en esta demo no hacemos uso del nombre del producto,
# pero se podría crear un diccionario de palabras
items.drop(['item_name'], axis=1, inplace=True)

In [21]:
items.head()

,item_id,item_category_id
0,0,40
1,1,76
2,2,40
3,3,40
4,4,40


# Análisis de Test

In [22]:
len(sales), len(test)

(2427355, 214200)

In [23]:
good_sales = test.reset_index().merge(sales, on=['item_id','shop_id'], how='left').dropna()
good_pairs = test[test.reset_index()['ID'].isin(good_sales['ID'])]
no_data_items = test[~(test['item_id'].isin(sales['item_id']))]

total = len(test)
caso1 = len(good_pairs)
caso2 = len(test)-len(no_data_items)-len(good_pairs)
caso3 = len(no_data_items)

print("From", total, "rows TEST:\n")
print('1. The item is at same shop in train:', caso1, round(caso1/total*100,2), "%")
print('2. Item exits in train but in another shop:', caso2, round(caso2/total*100,2), "%")
print('3. Thr item is completely new:', caso3, round(caso3/total*100,2), "%")

From 214200 rows TEST:

1. The item is at same shop in train: 111904 52.24 %
2. Item exits in train but in another shop: 86420 40.35 %
3. Thr item is completely new: 15876 7.41 %


---
# <center> UNIFY THE TABLES

# Matrix - Todos los productos para todas las tiendas y todos los meses
- Hay productos que no aparecen para algunos meses y tiendas, y se deben rellenar con cero
- Calcular la suma de ventas por mes - trabajamos por meses
- Recortar a 20 es una recondemación de los organizadores del concurso
- Rellenar a cero los valores nulos

In [24]:
sales.shop_id.nunique(), sales.item_id.nunique(), sales.date_block_num.nunique() 

(42, 20693, 34)

In [25]:
sales.shop_id.nunique() * sales.item_id.nunique() * sales.date_block_num.nunique() 

29549604

In [26]:
possibilities = 0

for month in range(34):

    sales_of_month = sales[sales.date_block_num==month]
    possibilities_of_month = sales_of_month.item_id.nunique() * sales_of_month.shop_id.nunique()
    
    print(month, possibilities_of_month)
    possibilities += possibilities_of_month
    
possibilities

0 264075
1 272628
2 279288
3 272664
4 285936
5 293558
6 291782
7 276353
8 269952
9 270507
10 274318
11 300200
12 264062
13 252434
14 256737
15 251440
16 249600
17 249120
18 252847
19 244278
20 234397
21 239153
22 244565
23 258505
24 237513
25 238333
26 239604
27 220006
28 211150
29 208895
30 213897
31 207460
32 207419
33 220920


8553596

In [27]:
sales_of_month

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,weekday,first_sale_day
2882335,2015-10-23,33,45,13315,649.0,1.0,4,963
2882336,2015-10-05,33,45,13880,229.0,1.0,0,662
2882337,2015-10-02,33,45,13881,659.0,1.0,4,1
2882338,2015-10-12,33,45,13881,659.0,1.0,0,1
2882339,2015-10-04,33,45,13923,169.0,1.0,6,3
...,...,...,...,...,...,...,...,...
2935844,2015-10-10,33,25,7409,299.0,1.0,5,1003
2935845,2015-10-09,33,25,7460,299.0,1.0,4,761
2935846,2015-10-14,33,25,7459,349.0,1.0,2,2
2935847,2015-10-22,33,25,7440,299.0,1.0,3,803


In [28]:

ts = time.time()
train = []
cols = ['date_block_num', 'shop_id', 'item_id']

for month in range(34):
    
    sales_of_month = sales[sales.date_block_num==month]
    
    shops_in_month = sales_of_month.shop_id.unique()
    items_in_month = sales_of_month.item_id.unique()
    
    print("Month", month, "\t", len(shops_in_month), "shops,", len(items_in_month), "items")
    
    train.append(np.array(list(product([month], shops_in_month, items_in_month)), dtype='int16'))
    
    
train = pd.DataFrame(np.vstack(train), columns=cols)

"""
matrix['date_block_num'] = matrix['date_block_num'].astype(np.int8)
matrix['shop_id']        = matrix['shop_id'].astype(np.int8)
matrix['item_id']        = matrix['item_id'].astype(np.int16)
matrix.sort_values(cols, inplace=True)
print(time.time() - ts, "segundos")
print("10.9 millones de filas:", len(matrix))

matrix.head()
"""
train

Month 0 	 35 shops, 7545 items
Month 1 	 36 shops, 7573 items
Month 2 	 36 shops, 7758 items
Month 3 	 36 shops, 7574 items
Month 4 	 37 shops, 7728 items
Month 5 	 37 shops, 7934 items
Month 6 	 37 shops, 7886 items
Month 7 	 37 shops, 7469 items
Month 8 	 37 shops, 7296 items
Month 9 	 37 shops, 7311 items
Month 10 	 37 shops, 7414 items
Month 11 	 38 shops, 7900 items
Month 12 	 38 shops, 6949 items
Month 13 	 38 shops, 6643 items
Month 14 	 39 shops, 6583 items
Month 15 	 40 shops, 6286 items
Month 16 	 40 shops, 6240 items
Month 17 	 40 shops, 6228 items
Month 18 	 41 shops, 6167 items
Month 19 	 41 shops, 5958 items
Month 20 	 41 shops, 5717 items
Month 21 	 41 shops, 5833 items
Month 22 	 41 shops, 5965 items
Month 23 	 41 shops, 6305 items
Month 24 	 41 shops, 5793 items
Month 25 	 41 shops, 5813 items
Month 26 	 41 shops, 5844 items
Month 27 	 41 shops, 5366 items
Month 28 	 41 shops, 5150 items
Month 29 	 41 shops, 5095 items
Month 30 	 41 shops, 5217 items
Month 31 	 41 shop

,date_block_num,shop_id,item_id
0,0,59,22154
1,0,59,2552
2,0,59,2554
3,0,59,2555
4,0,59,2564
...,...,...,...
8553591,33,21,7635
8553592,33,21,7638
8553593,33,21,7640
8553594,33,21,7632


In [29]:
train.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8553596 entries, 0 to 8553595
Data columns (total 3 columns):
 #   Column          Dtype
---  ------          -----
 0   date_block_num  int16
 1   shop_id         int16
 2   item_id         int16
dtypes: int16(3)
memory usage: 48.9 MB


In [30]:
#Add month sales (TARGET variable)
group = train.groupby(['date_block_num','shop_id','item_id']).agg({'item_cnt_day': ['sum']})
group.columns = ['item_cnt_month']
group.reset_index(inplace=True)
print("1.6 millones de filas:", len(group))
group.head()

KeyError: "Column 'item_cnt_day' does not exist!"

In [ ]:
matrix = pd.merge(matrix, group, on=cols, how='left')
matrix['item_cnt_month'] = (matrix['item_cnt_month']
                                .fillna(0)
                                .clip(0,20) # NB clip target here
                                .astype(np.float16))
matrix.head()

## Añadir test (mes 34)

In [ ]:
test['date_block_num'] = 34
test['date_block_num'] = test['date_block_num'].astype(np.int8)
test['shop_id']        = test['shop_id'].astype(np.int8)
test['item_id']        = test['item_id'].astype(np.int16)
matrix = pd.concat([matrix, test], ignore_index=True, sort=False, keys=cols)
matrix.fillna(0, inplace=True)

matrix.tail()

In [ ]:
len(matrix)

# <center> PARTE 3: Feature engineering
### Meter información de las otras tablas

In [ ]:
ts = time.time()
matrix = pd.merge(matrix, shops, on=['shop_id'], how='left')
matrix = pd.merge(matrix, items, on=['item_id'], how='left')
matrix = pd.merge(matrix, cats,  on=['item_category_id'], how='left')

matrix['shop_city']        = matrix['shop_city'].astype(np.int8)
matrix['shop_type']        = matrix['shop_type'].astype(np.int8)
matrix['item_category_id'] = matrix['item_category_id'].astype(np.int8)
matrix['item_type']        = matrix['item_type'].astype(np.int8)
matrix['item_subtype']     = matrix['item_subtype'].astype(np.int8)

# TODO: Borrar columna matrix['item_category_id']
time.time() - ts

In [ ]:
matrix.head()

In [ ]:
def megas(df):
    print(round(df.memory_usage(deep=True).sum() / 1024**2, 2), "MB")

In [ ]:
megas(matrix)

# Date features: Mes, nº días, nº findes
- Mes (1...12)
- Nº días del mes (28...31)
- Nº findes del mes (4 o 5)

In [ ]:
#matrix['month'] = matrix['date_block_num'] % 12
#days = pd.Series([31,28,31,30,31,30,31,31,30,31,30,31])
#matrix['days'] = matrix['month'].map(days).astype(np.int8)

In [ ]:
def count_days(date_block_num):
    year = 2013 + date_block_num // 12
    month = 1 + date_block_num % 12
    weeknd_count = len([1 for i in calendar.monthcalendar(year, month) if i[6] != 0])
    days_in_month = calendar.monthrange(year, month)[1]
    return month, days_in_month, weeknd_count

map_dict = {i: count_days(i) for i in range(35)}
map_dict

In [ ]:
matrix['month_id']      = matrix['date_block_num'].apply(lambda x: map_dict[x][0]).astype(np.int8)
matrix['month_days']    = matrix['date_block_num'].apply(lambda x: map_dict[x][1]).astype(np.int8)
matrix['month_weeknds'] = matrix['date_block_num'].apply(lambda x: map_dict[x][2]).astype(np.int8)

In [ ]:
matrix.head()

## Item: Es totalmente nuevo? (si=1, no=0) `item_isNew`

In [ ]:
# Para cada item, calcular el primer mes donde aparece
# E indicar con un 1 que el item es nuevo para ese mes
item_isNew_df = matrix.groupby(['item_id'])['date_block_num'].min().reset_index()
item_isNew_df['item_isNew'] = 1

matrix = pd.merge(matrix, item_isNew_df, on=['date_block_num', 'item_id'], how='left')

# Si no es nuevo se le pone 0
matrix['item_isNew'].fillna(0, inplace=True)
matrix['item_isNew'] = matrix['item_isNew'].astype('int8') 

## Item: Es nuevo en esa tienda? (si=1, no=0) `item_isNewOnShop`

In [ ]:
# Para cada item de cada tienda, calcular el primer mes donde aparece
# E indicar con un 1 que el item es nuevo para ese mes
item_isNewOnShop_df = matrix.groupby(['shop_id', 'item_id'])['date_block_num'].min().reset_index()
item_isNewOnShop_df['item_isNewOnShop'] = 1

matrix = pd.merge(matrix, item_isNewOnShop_df, on=['date_block_num', 'item_id', 'shop_id'], how='left')

# Si no es nuevo se le pone 0
matrix['item_isNewOnShop'].fillna(0, inplace=True)
matrix['item_isNewOnShop'] = matrix['item_isNewOnShop'].astype('int8') 

## Item: Duracion en meses `item_monthsDuration`

In [ ]:
# Para cada item, calcular el primer mes donde aparece
item_monthsDuration_df = matrix.groupby(['item_id'])['date_block_num'].min().reset_index()
item_monthsDuration_df.columns = ['item_id', 'item_monthsDuration']

matrix = pd.merge(matrix, item_monthsDuration_df, on=['item_id'], how='left')
matrix["item_monthsDuration"] = matrix["date_block_num"] - matrix["item_monthsDuration"]

matrix['item_monthsDuration'] = matrix['item_monthsDuration'].astype('int8') 

## Item en la tienda: Duracion en meses `itemShop_monthsDuration`

In [ ]:
# Para cada item, calcular el primer mes donde aparece
itemShop_monthsDuration_df = matrix.groupby(['shop_id', 'item_id'])['date_block_num'].min().reset_index()
itemShop_monthsDuration_df.columns = ['shop_id', 'item_id', 'itemShop_monthsDuration']

matrix = pd.merge(matrix, itemShop_monthsDuration_df, on=['shop_id', 'item_id'], how='left')
matrix["itemShop_monthsDuration"] = matrix["date_block_num"] - matrix["itemShop_monthsDuration"]

matrix['itemShop_monthsDuration'] = matrix['itemShop_monthsDuration'].astype('int8') 

## Nº meses desde la primera venta por (producto y tienda), y por (producto)
### NO SE SI HAY FUAGA DE DATO AQUI, PENSARLO BIEN

In [ ]:
vendidos=matrix[matrix.item_cnt_month>0]

# Para cada item
item_firstSale_df = vendidos.groupby(['item_id'])['date_block_num'].min().reset_index()
item_firstSale_df.columns = ['item_id', 'item_firstSale']

matrix = pd.merge(matrix, item_firstSale_df, on=['item_id'], how='left')
matrix["item_firstSale"] = matrix["date_block_num"] - matrix["item_firstSale"]

matrix['item_firstSale'].fillna(-1, inplace=True)
matrix['item_firstSale'] = matrix['item_firstSale'].astype('int8') 



itemShop_firstSale_df = vendidos.groupby(['item_id'])['date_block_num'].min().reset_index()
itemShop_firstSale_df.columns = ['item_id', 'itemShop_firstSale']

matrix = pd.merge(matrix, itemShop_firstSale_df, on=['item_id'], how='left')
matrix["itemShop_firstSale"] = matrix["date_block_num"] - matrix["itemShop_firstSale"]

matrix['itemShop_firstSale'].fillna(-1, inplace=True)
matrix['itemShop_firstSale'] = matrix['itemShop_firstSale'].astype('int8') 

# Nº meses desde la última venta por producto y tienda, y por producto
## 45 MINUNTOS TARDA ESTO


In [ ]:
from tqdm import tqdm

In [ ]:
ts = time.time()
cache = {}
matrix['item_last_sale'] = -1
matrix['item_last_sale'] = matrix['item_last_sale'].astype(np.int8)
for idx, row in tqdm(matrix.iterrows(), total=len(matrix)):    
    key = row.item_id  #solo por producto
    if key not in cache:
        if row.item_cnt_month!=0:
            cache[key] = row.date_block_num
    else:
        last_date_block_num = cache[key]
        if row.date_block_num>last_date_block_num:
            matrix.at[idx, 'item_last_sale'] = row.date_block_num - last_date_block_num
            cache[key] = row.date_block_num            
time.time() - ts

In [ ]:
ts = time.time()
cache = {}
matrix['item_shop_last_sale'] = -1
matrix['item_shop_last_sale'] = matrix['item_shop_last_sale'].astype(np.int8)
for idx, row in tqdm(matrix.iterrows(), total=len(matrix)):    
    key = str(row.item_id)+' '+str(row.shop_id) #por producto y tienda
    if key not in cache:
        if row.item_cnt_month!=0:
            cache[key] = row.date_block_num
    else:
        last_date_block_num = cache[key]
        matrix.at[idx, 'item_shop_last_sale'] = row.date_block_num - last_date_block_num
        cache[key] = row.date_block_num
time.time() - ts

# <center> PARTE 4: Feat Eng con Lags

## Crear variables históricas ventas de meses anteriores

In [ ]:
def lag_feature(df, lags, col):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id', col+'_lag_'+str(i)]
        shifted['date_block_num'] += i # ADELANTO EL MES PARA LUEGO HACER MERGE Y QUE SE PRODUZCA EL LAG
        df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df

In [ ]:
ts = time.time()
matrix = lag_feature(matrix, [1,2,3,6,12], 'item_cnt_month')
time.time() - ts

In [ ]:
matrix[matrix.date_block_num==0].head()

In [ ]:
matrix[matrix.date_block_num==33].head()

In [ ]:
megas(matrix)

# Variables agrupadas (cuidado con la fuga de datos)
La fuga de datos puede hacer que cree modelos predictivos demasiado optimistas, pero realmente son completamente inválidos.

La fuga de datos se produce cuando se utiliza información de fuera del conjunto de datos de capacitación para crear el modelo. Esta información adicional puede permitir que el modelo aprenda o sepa algo que de otro modo no sabría y, a su vez, invalidará el rendimiento estimado del modo que se está construyendo.

In [ ]:
matrix.head()

In [ ]:
def mean_of_item_cnt_by(groupby_cols, lags, new_col_name, df):
    ts = time.time()
    
    # Crear variable
    group = df.groupby(groupby_cols).agg({'item_cnt_month': ['mean']})
    group.columns = [ new_col_name ]
    group.reset_index(inplace=True)
    
    # Meter variable en los datos
    df = pd.merge(df, group, on=groupby_cols, how='left')
    df[new_col_name] = df[new_col_name].astype(np.float16)
    
    # Hacer lag de la variable (se evita FUGA DE DATOS)
    df = lag_feature(df, lags, new_col_name)
    df.drop([new_col_name], axis=1, inplace=True)

    print(round(time.time() - ts, 2), "segundos")
    return df

In [ ]:
# La media Nº de ventas agrupado por mes
matrix = mean_of_item_cnt_by(groupby_cols=['date_block_num'],
                    lags=[1],
                    new_col_name="date_avg_item_cnt",
                    df=matrix)

In [ ]:
# La media Nº de ventas agrupado por mes y producto
matrix = mean_of_item_cnt_by(groupby_cols=['date_block_num', 'item_id'],
                    lags=[1,2,3,6,12],
                    new_col_name="date_item_avg_item_cnt",
                    df=matrix)

In [ ]:
# La media Nº de ventas agrupado por mes y tienda
matrix = mean_of_item_cnt_by(groupby_cols=['date_block_num', 'shop_id'],
                    lags=[1,2,3,6,12],
                    new_col_name="date_shop_avg_item_cnt",
                    df=matrix)

In [ ]:
# La media Nº de ventas agrupado por mes y categoría
matrix = mean_of_item_cnt_by(groupby_cols=['date_block_num', 'item_category_id'],
                    lags=[1],
                    new_col_name="date_cat_avg_item_cnt",
                    df=matrix)

In [ ]:
# La media Nº de ventas agrupado por mes, tienda y categoría
matrix = mean_of_item_cnt_by(groupby_cols=['date_block_num', 'shop_id', 'item_category_id'],
                    lags=[1],
                    new_col_name="date_shop_cat_avg_item_cnt",
                    df=matrix)

In [ ]:
# La media Nº de ventas agrupado por mes, tienda y tipo
matrix = mean_of_item_cnt_by(groupby_cols=['date_block_num', 'shop_id', 'type_code'],
                    lags=[1],
                    new_col_name="date_shop_type_avg_item_cnt",
                    df=matrix)

In [ ]:
# La media Nº de ventas agrupado por mes, tienda y subtipo
matrix = mean_of_item_cnt_by(groupby_cols=['date_block_num', 'shop_id', 'subtype_code'],
                    lags=[1],
                    new_col_name="date_shop_subtype_avg_item_cnt",
                    df=matrix)

In [ ]:
# La media Nº de ventas agrupado por mes y ciudad
matrix = mean_of_item_cnt_by(groupby_cols=['date_block_num', 'city_code'],
                    lags=[1],
                    new_col_name="date_city_avg_item_cnt",
                    df=matrix)

In [ ]:
# La media Nº de ventas agrupado por mes, producto y ciudad
matrix = mean_of_item_cnt_by(groupby_cols=['date_block_num', 'item_id', 'city_code'],
                    lags=[1],
                    new_col_name="date_item_city_avg_item_cnt",
                    df=matrix)

In [ ]:
# La media Nº de ventas agrupado por mes y tipo
matrix = mean_of_item_cnt_by(groupby_cols=['date_block_num', 'type_code'],
                    lags=[1],
                    new_col_name="date_type_avg_item_cnt",
                    df=matrix)

In [ ]:
# La media Nº de ventas agrupado por mes y subtipo
matrix = mean_of_item_cnt_by(groupby_cols=['date_block_num', 'subtype_code'],
                    lags=[1],
                    new_col_name="date_subtype_avg_item_cnt",
                    df=matrix)

In [ ]:
matrix.tail(7).T # #matrix[matrix.date_block_num > 11].head().T

In [ ]:
megas(matrix)

# Tendencia de precios en los últimos seis meses.
### Delta Precio = (Precio Medio Producto agrupado por mes - Precio Medio Producto) / Precio Medio Producto

In [ ]:
ts = time.time()
group = train.groupby(['item_id']).agg({'item_price': ['mean']})
group.columns = ['item_avg_item_price']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['item_id'], how='left')
matrix['item_avg_item_price'] = matrix['item_avg_item_price'].astype(np.float16)

group = train.groupby(['date_block_num','item_id']).agg({'item_price': ['mean']})
group.columns = ['date_item_avg_item_price']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','item_id'], how='left')
matrix['date_item_avg_item_price'] = matrix['date_item_avg_item_price'].astype(np.float16)

lags = [1,2,3,4,5,6]
matrix = lag_feature(matrix, lags, 'date_item_avg_item_price')

for i in lags:
    matrix['delta_price_lag_'+str(i)] = \
        (matrix['date_item_avg_item_price_lag_'+str(i)] - matrix['item_avg_item_price']) / matrix['item_avg_item_price']

#primer histórico distinto de cero, partimos que no tenemos el precio de los productos para el mes de validación   
def select_trend(row):
    for i in lags:
        if row['delta_price_lag_'+str(i)]:
            return row['delta_price_lag_'+str(i)]
    return 0
    
matrix['delta_price_lag'] = matrix.apply(select_trend, axis=1)
matrix['delta_price_lag'] = matrix['delta_price_lag'].astype(np.float16)
matrix['delta_price_lag'].fillna(0, inplace=True)

fetures_to_drop = ['item_avg_item_price', 'date_item_avg_item_price']
for i in lags:
    fetures_to_drop += ['date_item_avg_item_price_lag_'+str(i)]
    fetures_to_drop += ['delta_price_lag_'+str(i)]

matrix.drop(fetures_to_drop, axis=1, inplace=True)
time.time() - ts

In [ ]:
matrix.head()

In [ ]:
megas(matrix)

# Tendencia de ingresos de la tienda del mes pasado

In [ ]:
ts = time.time()
train['revenue'] = train['item_price'] * train['item_cnt_day'] # ingresos precio * ventas

group = train.groupby(['date_block_num','shop_id']).agg({'revenue': ['sum']})
group.columns = ['date_shop_revenue']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','shop_id'], how='left')
matrix['date_shop_revenue'] = matrix['date_shop_revenue'].astype(np.float32)


group = group.groupby(['shop_id']).agg({'date_shop_revenue': ['mean']})
group.columns = ['shop_avg_revenue']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['shop_id'], how='left')
matrix['shop_avg_revenue'] = matrix['shop_avg_revenue'].astype(np.float32)

matrix['delta_revenue'] = (matrix['date_shop_revenue'] - matrix['shop_avg_revenue']) / matrix['shop_avg_revenue']
matrix['delta_revenue'] = matrix['delta_revenue'].astype(np.float16)

matrix = lag_feature(matrix, [1], 'delta_revenue')

matrix.drop(['date_shop_revenue','shop_avg_revenue','delta_revenue'], axis=1, inplace=True)
time.time() - ts

In [ ]:
matrix.head()

# Quitamos los primeres 12 meses
### No tienen valores históricos

In [ ]:
matrix = matrix[matrix.date_block_num > 11]

# Rellenamos a cero posibles valores a nulo

In [ ]:
ts = time.time()
def fill_na(df):
    for col in df.columns:
        if ('_lag_' in col) & (df[col].isnull().any()):
            if ('item_cnt' in col):
                df[col].fillna(0, inplace=True)         
    return df

matrix = fill_na(matrix)
time.time() - ts

In [ ]:
matrix.head()

In [ ]:
matrix.columns

In [ ]:
matrix.info()

In [ ]:
matrix.sample(10).T

# Guardamos los datos

In [ ]:
ts = time.time()
matrix.to_pickle('../../predicFutureSales_data.pkl')
time.time() - ts

In [ ]:
"""
del matrix
del group
del items
del shops
del cats
del train
del cache

gc.collect();
"""

# Separación de datos de entrenamiento y validación

In [ ]:
ts = time.time()

df = pd.read_pickle('../../predicFutureSales_data.pkl')
X_train = df[df.date_block_num <  33].drop(['item_cnt_month'], axis=1)
Y_train = df[df.date_block_num <  33]['item_cnt_month']
X_valid = df[df.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = df[df.date_block_num == 33]['item_cnt_month']
X_test  = df[df.date_block_num == 34].drop(['item_cnt_month'], axis=1)

time.time() - ts

# Entrenamiento

In [ ]:
ts = time.time()

params = {
    'objective': 'mse',
    'metric': 'rmse',
    'num_leaves': 2 ** 7 - 1,
    'learning_rate': 0.005,
    'feature_fraction': 0.75,
    'bagging_fraction': 0.75,
    'bagging_freq': 5,
    'seed': 1,
    'verbose': 1
}

lgb_train = lgb.Dataset(X_train, Y_train)
lgb_eval  = lgb.Dataset(X_valid, Y_valid, reference=lgb_train)

evals_result = {}
model = lgb.train(
        params, 
        lgb_train,
        num_boost_round=3000,
        valid_sets=(lgb_train, lgb_eval), 
        verbose_eval=5, 
        evals_result = evals_result,
        early_stopping_rounds = 100)
time.time() - ts

In [ ]:
lgb.plot_importance(
    model, 
    max_num_features=50, 
    importance_type='gain', 
    figsize=(12,10));

# Entrenar con todo

In [ ]:
X_train = df[df.date_block_num < 34].drop(['item_cnt_month'], axis=1)
Y_train = df[df.date_block_num < 34]['item_cnt_month']

In [ ]:
ts = time.time()

params = {
    'objective': 'mse',
    'metric': 'rmse',
    'num_leaves': 2 ** 7 - 1,
    'learning_rate': 0.005,
    'feature_fraction': 0.75,
    'bagging_fraction': 0.75,
    'bagging_freq': 5,
    'seed': 1,
    'verbose': 1
}

lgb_train = lgb.Dataset(X_train, Y_train)
lgb_eval  = lgb.Dataset(X_valid, Y_valid, reference=lgb_train)

evals_result = {}
model = lgb.train(
        params, 
        lgb_train,
        num_boost_round=3000,
        valid_sets=(lgb_train, lgb_eval), 
        verbose_eval=5, 
        evals_result = evals_result,
        early_stopping_rounds = 100)
time.time() - ts

In [ ]:
Y_test_pred = model.predict(X_test).clip(0, 20)
Y_test_pred

In [ ]:
sub["item_cnt_month"] = Y_test_pred
sub

In [ ]:
sub.to_csv('../../segundoModelo_submission.csv')

In [ ]:
ts = time.time()
'''
#0.7875788072832439
params = {'bagging_fraction': 0.99, 'bagging_freq': 8, 'bagging_seed': SEED, 
          'feature_fraction': 0.6802441991432773, 'feature_fraction_seed': SEED, 
          'lambda_l1': 200, 'lambda_l2': 200, 'learning_rate': 0.015, 'max_bin': 176, 
          'metric': 'rmse', 'min_child_samples': 656,
          'min_child_weight': 636, 'min_split_gain': 0.1512409762978552, 'nthread': 8, 
          'num_leaves': 454, 'objective': 'mse', 'verbose': 0}
'''
SEED = 3246584
#0.7838050006725876
params = {'bagging_fraction': 0.5923685197548111, 'bagging_freq': 8, 'bagging_seed': SEED, 
          'feature_fraction': 0.6471558662772907, 'feature_fraction_seed': SEED, 
          'lambda_l1': 8, 'lambda_l2': 454, 'learning_rate': 0.15007983998456081, 'max_bin': 899, 
          'metric': 'rmse', 'min_child_samples': 463,
          'min_child_weight': 578, 'min_split_gain': 0.21257474291512948, 'nthread': 8, 
          'num_leaves': 278, 'objective': 'mse', 'verbose': 0}




evals_result = {}
model = lgb.train(params, 
            train_set=lgb.Dataset(X_train, label=Y_train), 
            num_boost_round=50000,
            valid_sets=[lgb.Dataset(X_train, label=Y_train), lgb.Dataset(X_valid, label=Y_valid)] ,
            valid_names=['train', 'val'],
            evals_result=evals_result,
            verbose_eval=50,
            early_stopping_rounds=400)
time.time() - ts

In [ ]:
print('Plotting metrics recorded during training...')
ax = lgb.plot_metric(evals_result, metric='rmse')
plt.show()

In [ ]:
print('Plotting feature importances...')
ax = lgb.plot_importance(model, max_num_features=15)
plt.show()

In [ ]:
ax = lgb.plot_tree(model, tree_index=0, figsize=(20, 8), show_info=['split_gain'])
plt.show()

In [ ]:
for i in range(4):
    graph = lgb.create_tree_digraph(model, tree_index=i, name='./trees/Tree' + str(i))
    graph.render(view=False)

# Predicción

In [ ]:
Y_valid_predict = model.predict(X_valid, num_iteration=model.best_iteration).clip(0, 20)

rmseAux = mean_squared_error(Y_valid, Y_valid_predict)
print (rmseAux)

submission = pd.DataFrame({
    "shop_id": X_valid['shop_id'],
    "item_id": X_valid['item_id'], 
    "item_cnt_month": Y_valid,
    "predict": Y_valid_predict,
    "predict_round": Y_valid_predict.round(),
    "difference": (Y_valid - Y_valid_predict.round())
})
submission.to_csv('lgb_submission.csv', index=False)

In [ ]:
submission['difference'].describe()

In [ ]:
print ("Nº registros: ", submission['difference'].count())
print ("Nº aciertos: ", submission[submission.difference == 0]['difference'].count())

In [ ]:
submission[submission.item_cnt_month > 3].sample(20, random_state =3333) #101, 5555, 3333

In [ ]:
ax = sns.distplot(submission['difference'])

In [ ]:
ax = sns.distplot(submission[submission.shop_id == 2]['difference'])

In [ ]:
ax = sns.distplot(submission[submission.item_id == 30]['difference'])

In [ ]:
shops = sorted(submission['shop_id'].unique().tolist())

@interact(shopid=shops)
def DiffShops(shopid):
     ax = sns.distplot(submission[submission.shop_id == shopid]['difference'])

In [ ]:
items = sorted(submission['item_id'].unique().tolist())

@interact(itemid=items)
def DiffItems(itemid):
     ax = sns.distplot(submission[submission.item_id == itemid]['difference'])